In [ ]:
# 1. 패키지 설치
!pip install pandas numpy ensemble-boxes scikit-learn scipy

# 2. Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 3. 필요한 라이브러리 임포트 및 버전 확인
import pandas as pd
import numpy as np
import glob
from ensemble_boxes import weighted_boxes_fusion
import sys  # sys 모듈을 임포트하여 sys.exit() 사용 가능


In [ ]:
# 4. PredictionString 파싱 함수 정의
def parse_prediction_string(pred_str, image_width, image_height):
    labels = []
    scores = []
    boxes = []
    if pred_str.strip() == '':
        return labels, scores, boxes
    parts = pred_str.strip().split()
    for i in range(0, len(parts), 6):
        try:
            label = int(parts[i])
            score = float(parts[i+1])
            xmin = float(parts[i+2]) / image_width  # 정규화
            ymin = float(parts[i+3]) / image_height  # 정규화
            xmax = float(parts[i+4]) / image_width  # 정규화
            ymax = float(parts[i+5]) / image_height  # 정규화
            labels.append(label)
            scores.append(score)
            boxes.append([xmin, ymin, xmax, ymax])
        except (IndexError, ValueError) as e:
            print(f'Error parsing PredictionString: {e}')
            continue
    return labels, scores, boxes


In [ ]:
# 5. CSV 파일들이 저장된 디렉토리 경로 설정
csv_dir = '/content/drive/MyDrive/ensemble/'  # 실제 경로로 변경

In [ ]:
# 6. 이미지 크기 정의 (필요에 따라 변경)
image_width = 1024  # 예시 이미지의 너비
image_height = 1024  # 예시 이미지의 높이

# 7. CSV 파일 경로 설정 및 읽기
csv_files = glob.glob(csv_dir + '*.csv')  # 패턴에 맞게 경로 수정

predictions = {}

for csv_file in csv_files:
    try:
        df = pd.read_csv(csv_file, dtype={'PredictionString': str, 'image_id': str}, engine='python', on_bad_lines='skip')
        print(f'Reading {csv_file} - {len(df)} rows')
    except Exception as e:
        print(f'Error reading {csv_file}: {e}')
        continue

    for idx, row in df.iterrows():
        image_id = row.get('image_id', '').strip()
        pred_str = row.get('PredictionString', '')

        if not image_id:
            print(f'Empty image_id at row {idx} in {csv_file}')
            continue  # Skip rows with empty image_id

        # pred_str이 문자열인지 확인하고, NaN이나 float 처리
        if not isinstance(pred_str, str):
            pred_str = ''  # NaN이나 float인 경우 빈 문자열로 처리

        labels, scores, boxes = parse_prediction_string(pred_str, image_width, image_height)

        if image_id not in predictions:
            predictions[image_id] = {'labels': [], 'scores': [], 'boxes': []}

        predictions[image_id]['labels'].append(labels)
        predictions[image_id]['scores'].append(scores)
        predictions[image_id]['boxes'].append(boxes)

print("CSV 파일 처리 완료.")

Reading /content/drive/MyDrive/ensemble/atss_swin_l_sr_6423.csv - 4871 rows
Reading /content/drive/MyDrive/ensemble/co_dino_stages0_1_freeze_6492.csv - 4871 rows
Reading /content/drive/MyDrive/ensemble/salience_full_constrative_6645.csv - 4871 rows
Reading /content/drive/MyDrive/ensemble/dino_6743.csv - 4871 rows
Reading /content/drive/MyDrive/ensemble/dino_full_6828.csv - 4871 rows
Reading /content/drive/MyDrive/ensemble/co_dino_lsj_6513.csv - 4871 rows
Reading /content/drive/MyDrive/ensemble/co_dino_6666.csv - 4871 rows
CSV 파일 처리 완료.


In [ ]:
# 모델별 성능 점수 정의
model_scores = [
    0.6423,  # 1번 모델: atss_swin_l_sr_6423.csv
    # 0.6087,  # 2번 모델: dino_random_split_6087.csv
    # 0.6153,  # 3번 모델: relation_6153.csv
    0.6492,  # 4번 모델: co_dino_stages0_1_freeze_6492.csv
    # 0.6862,  # 6번 모델: ensemble_6862.csv
    # 0.6320,  # 7번 모델: co_dino_stages0_freeze_6320.csv
    0.6645,   # 8번 모델: salience_full_constrative_6645.csv
    0.6743,    # 9번 모델: dino_6743.csv
    0.6828,     # 10번 모델: dino_full_6828.csv
    0.6513,
    0.6666
]

# 가중치 정규화 (가중치의 합이 1이 되도록)
total_score = sum(model_scores)
weights = [score / total_score for score in model_scores]

# WBF 수행 및 결과 수집
ensemble_results_final = []

for image_id, preds in predictions.items():
    # 각 모델의 boxes, scores, labels
    boxes_list = preds['boxes']      # List of lists
    scores_list = preds['scores']    # List of lists
    labels_list = preds['labels']    # List of lists

    try:
        # Weighted Boxes Fusion 수행
        boxes, scores, labels = weighted_boxes_fusion(
            boxes_list,
            scores_list,
            labels_list,
            weights=weights,                  # 수정된 가중치 적용
            iou_thr=0.8,
            skip_box_thr=0.0,
            conf_type='max'                   # conf_type을 'avg'로 변경하여 성능 차이 반영
        )
    except Exception as e:
        print(f'Error during WBF for image_id {image_id}: {e}')
        continue  # 해당 이미지를 건너뜀

    # PredictionString 형식으로 변환
    pred_strings = []
    for label, score, box in zip(labels, scores, boxes):
        xmin, ymin, xmax, ymax = box
        # 원래 좌표로 복원 (image_width와 image_height가 정의되어 있어야 함)
        xmin_orig = xmin * image_width
        ymin_orig = ymin * image_height
        xmax_orig = xmax * image_width
        ymax_orig = ymax * image_height
        pred_strings.extend([
            str(int(label)),
            f"{score:.3f}",
            f"{xmin_orig:.2f}",
            f"{ymin_orig:.2f}",
            f"{xmax_orig:.2f}",
            f"{ymax_orig:.2f}"
        ])

    pred_string = ' '.join(pred_strings)
    ensemble_results_final.append({'image_id': image_id, 'PredictionString': pred_string})

print('Weighted Boxes Fusion completed.')


Weighted Boxes Fusion completed.


In [ ]:
# 9. 앙상블 결과를 DataFrame으로 변환 및 열 순서 변경
ensemble_df = pd.DataFrame(ensemble_results_final)

# 열 순서 변경: 'PredictionString' 먼저, 'image_id' 나중
ensemble_df = ensemble_df[['PredictionString', 'image_id']]

# 중복된 image_id가 있는지 확인하고 제거 (필요 시)
ensemble_df = ensemble_df.drop_duplicates(subset=['image_id'])

# 10. CSV 파일로 저장
output_csv_path = '/content/drive/MyDrive/ensemble/final_ensemble_result2.csv'  # 저장할 경로로 변경
ensemble_df.to_csv(output_csv_path, index=False)

print(f'앙상블된 결과가 {output_csv_path}에 저장되었습니다.')

# 11. 결과 확인
# 파일의 처음 몇 줄을 텍스트로 읽어보기
print("결과 CSV의 처음 10줄:")
with open(output_csv_path, 'r') as f:
    for _ in range(10):
        print(f.readline().strip())

앙상블된 결과가 /content/drive/MyDrive/ensemble/final_ensemble_result2.csv에 저장되었습니다.
결과 CSV의 처음 10줄:
PredictionString,image_id
7 0.957 215.96 50.11 454.62 473.49 7 0.955 603.05 517.52 956.53 1023.53 7 0.944 449.51 600.83 647.24 875.02 7 0.939 118.71 421.36 331.40 720.68 7 0.935 390.05 191.78 600.41 545.53 7 0.930 651.03 333.41 880.43 608.41 1 0.917 567.02 103.53 743.12 360.05 7 0.912 546.27 343.50 708.09 605.25 7 0.874 255.87 410.28 433.68 592.89 5 0.821 344.00 641.10 395.22 725.37 2 0.794 329.17 639.09 353.28 683.56 7 0.768 266.30 431.29 432.50 618.83 5 0.744 244.31 684.46 301.48 748.19 6 0.716 431.31 465.68 564.96 682.39 7 0.693 540.04 185.67 574.52 239.89 5 0.664 467.32 552.23 559.77 598.84 5 0.605 323.35 569.21 380.48 619.22 5 0.571 324.12 615.74 469.58 742.34 1 0.548 631.51 255.58 782.55 374.05 7 0.527 267.35 433.06 430.94 580.25 5 0.523 328.97 639.21 353.77 683.63 5 0.522 318.80 680.18 346.79 743.53 6 0.518 244.53 683.97 301.36 747.90 5 0.512 431.18 466.00 564.56 680.12 5 0.511 387.08 6

In [ ]:
# 이미지별 bounding box 예측 개수 계산
image_bbox_counts = []

total_bbox_count = 0  # 전체 예측된 bounding box 개수

for image_id, preds in predictions.items():
    # 각 모델이 예측한 bounding box 개수 합계 계산
    total_boxes = sum(len(boxes) for boxes in preds['boxes'])  # 예측된 box 개수 합계
    total_bbox_count += total_boxes  # 전체 bbox 개수 누적
    image_bbox_counts.append({'image_id': image_id, 'bbox_count': total_boxes})

# 전체 이미지 중에서 가장 많은 bounding box가 예측된 이미지 찾기
max_bbox_count = max(image_bbox_counts, key=lambda x: x['bbox_count'])

# 가장 많은 bounding box가 예측된 이미지의 개수 출력
print(f"가장 많은 bounding box가 예측된 이미지 ID: {max_bbox_count['image_id']}")
print(f"해당 이미지의 예측된 bounding box 개수: {max_bbox_count['bbox_count']}")

# 전체 예측된 bounding box 개수 출력
print(f"전체 이미지에서 예측된 bounding box 개수: {total_bbox_count}")


가장 많은 bounding box가 예측된 이미지 ID: test/1731.jpg
해당 이미지의 예측된 bounding box 개수: 1863
전체 이미지에서 예측된 bounding box 개수: 7293232
